In [10]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td

def cifar_loader(batch_size, shuffle_test=False):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                       std=[0.225, 0.225, 0.225])
    train = datasets.CIFAR10('./data', train=True, download=True,
        transform=transforms.Compose([transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, 4), transforms.ToTensor(), normalize]))
    test = datasets.CIFAR10('./data', train=False,
        transform=transforms.Compose([transforms.ToTensor(), normalize]))
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
        shuffle=True, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=shuffle_test, pin_memory=True) 
    return train_loader, test_loader



In [11]:
batch_size = 64
test_batch_size = 64
input_size = 3072
N = batch_size
D_in = input_size
H = 50
D_out = 10
num_epochs = 10


train_loader, _ = cifar_loader(batch_size)
_, test_loader = cifar_loader(test_batch_size)


class MultiLayerFCNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(MultiLayerFCNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, H)
        self.linear3 = torch.nn.Linear(H, H)
        self.linear4 = torch.nn.Linear(H, D_out)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = self.linear4(x)
        return F.log_softmax(x)


Files already downloaded and verified
Files already downloaded and verified


In [13]:
# convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = dataiter.next()
features, labels = data
print(features, labels)

tensor([[[[-2.1556, -2.1556, -2.1556,  ..., -2.1556, -2.1556, -2.1556],
          [-2.1556, -2.1556, -2.1556,  ..., -2.1556, -2.1556, -2.1556],
          [-2.1556, -2.1556, -2.1556,  ..., -2.1556, -2.1556, -2.1556],
          ...,
          [-2.1556, -2.1556, -2.1556,  ..., -1.0227, -0.9878, -1.0749],
          [-2.1556, -2.1556, -2.1556,  ..., -0.9181, -0.9529, -0.9355],
          [-2.1556, -2.1556, -2.1556,  ..., -0.8484, -0.9355, -0.9181]],

         [[-2.0267, -2.0267, -2.0267,  ..., -2.0267, -2.0267, -2.0267],
          [-2.0267, -2.0267, -2.0267,  ..., -2.0267, -2.0267, -2.0267],
          [-2.0267, -2.0267, -2.0267,  ..., -2.0267, -2.0267, -2.0267],
          ...,
          [-2.0267, -2.0267, -2.0267,  ..., -0.7718, -0.7369, -0.8241],
          [-2.0267, -2.0267, -2.0267,  ..., -0.6672, -0.7195, -0.6846],
          [-2.0267, -2.0267, -2.0267,  ..., -0.5975, -0.6846, -0.6672]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, -

In [10]:
model = MultiLayerFCNet(D_in, H, D_out)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [11]:
for epoch in range(num_epochs):
    avg_loss_epoch = 0
    batch_loss = 0
    total_batches = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 32 * 32 * 3)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_batches += 1
        batch_loss += loss.item()
    avg_loss_epoch = batch_loss / total_batches
    print('Epoch [{}/{}], Averge Loss:for epoch[{}, {:.4f}]'.format(epoch + 1, num_epochs, epoch + 1, avg_loss_epoch))

/var/folders/l9/pkrn89011n744pxf41ndfjrr0000gn/T/ipykernel_99563/4241812732.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Epoch [1/10], Averge Loss:for epoch[1, 1.8988]
Epoch [2/10], Averge Loss:for epoch[2, 1.7276]
Epoch [3/10], Averge Loss:for epoch[3, 1.6692]
Epoch [4/10], Averge Loss:for epoch[4, 1.6368]
Epoch [5/10], Averge Loss:for epoch[5, 1.6183]
Epoch [6/10], Averge Loss:for epoch[6, 1.5916]
Epoch [7/10], Averge Loss:for epoch[7, 1.5741]
Epoch [8/10], Averge Loss:for epoch[8, 1.5615]
Epoch [9/10], Averge Loss:for epoch[9, 1.5529]
Epoch [10/10], Averge Loss:for epoch[10, 1.5457]


In [12]:
correct = 0
total = 0
for images, labels in test_loader:
    images = images.reshape(-1, 3 * 32 * 32)
    outputs_test = model(images)
    _, predicted = torch.max(outputs_test.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 46 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 44 %
Accuracy of the network on the 10000 test images: 42 %
Accuracy of the network on the 10000 test images: 42 %
Accuracy of the network on the 10000 test images: 42 %
Accuracy of the network on the 10000 test images: 43 %
Accuracy of the network on the 10000 test images: 43 %
Accuracy of the network on the 10000 test images: 43 %
Accuracy of the network on the 10000 test images: 44 %
Accuracy of the network on the 10000 test images: 44 %
Accuracy of the network on the 10000 test images: 44 %
Accuracy of the network on the 10000 test images: 43 %
Accuracy of the network on the 10000 test images: 44 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 46 %
Accuracy o

/var/folders/l9/pkrn89011n744pxf41ndfjrr0000gn/T/ipykernel_99563/4241812732.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 44 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy of the network on the 10000 test images: 45 %
Accuracy o